In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as tts

from sklearn.ensemble import RandomForestRegressor

In [2]:
#reading train data
df = pd.read_csv('./train_NIR5Yl1.csv')

In [3]:
#dropping ID feature
df.drop('ID',axis=1,inplace=True)

In [4]:
#dropping username
df.drop('Username',axis=1,inplace=True)

In [5]:
#one hot encodeing the data
df = pd.get_dummies(df)

In [6]:
# dropping the Tag_x feature as it can be predicted 
# by rest of the one hot encoded Tags
df.drop('Tag_x',axis=1,inplace=True)

In [7]:
#dividing data into features and labels
X = df.drop('Upvotes',axis=1)
y = df['Upvotes']
print(X.shape)
print(y.shape)

(330045, 12)
(330045,)


In [8]:
#dividing data into train and test
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.1,random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(297040, 12)
(297040,)
(33005, 12)
(33005,)


### TUNING

In [17]:
#from sklearn.model_selection import GridSearchCV

#reg = RandomForestRegressor(random_state=42)

#parameter grid
#params = {
 #   'max_depth' : [5,6],
 #   'n_estimators' : [50,75],
#}
#performing grid serach
#estimator = GridSearchCV(reg,
#                  params,
#                  cv=4,
#                  scoring="neg_mean_squared_error",
#                  n_jobs=1,
#                  verbose=2)

#estimator.fit(X_train,y_train)

#printing the best parameters
#print(estimator.best_params_)

#printing scores
#print(np.sqrt(-1*estimator.best_score_))

Fitting 4 folds for each of 4 candidates, totalling 16 fits
[CV] max_depth=5, n_estimators=50 ....................................
[CV] ..................... max_depth=5, n_estimators=50, total=   9.9s
[CV] max_depth=5, n_estimators=50 ....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.2s remaining:    0.0s


[CV] ..................... max_depth=5, n_estimators=50, total=  10.7s
[CV] max_depth=5, n_estimators=50 ....................................
[CV] ..................... max_depth=5, n_estimators=50, total=  10.2s
[CV] max_depth=5, n_estimators=50 ....................................
[CV] ..................... max_depth=5, n_estimators=50, total=  10.3s
[CV] max_depth=5, n_estimators=75 ....................................
[CV] ..................... max_depth=5, n_estimators=75, total=  15.5s
[CV] max_depth=5, n_estimators=75 ....................................
[CV] ..................... max_depth=5, n_estimators=75, total=  15.7s
[CV] max_depth=5, n_estimators=75 ....................................
[CV] ..................... max_depth=5, n_estimators=75, total=  15.3s
[CV] max_depth=5, n_estimators=75 ....................................
[CV] ..................... max_depth=5, n_estimators=75, total=  15.3s
[CV] max_depth=6, n_estimators=50 ....................................
[CV] .

[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:  3.8min finished


{'max_depth': 6, 'n_estimators': 75}
1417.5150305674363


In [ ]:
# tried the above parameters but the results were not good

#clf = RandomForestRegressor(max_depth=6,n_estimators = 75)
#clf.fit(X,y)

In [15]:
#preparing the model
clf = RandomForestRegressor(random_state = 1)
#fitting the model on entire data
clf.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [16]:
#calculating score
from sklearn.metrics import mean_squared_error
preds = clf.predict(X_test)
score = np.sqrt(mean_squared_error(y_test,preds))
print(score)

1012.2778368282012


### MAKING PREDICTIONS ON TEST FILE

In [17]:
#reading the test data
test = pd.read_csv('./test_8i3B3FC.csv')

In [18]:
#preparing the test data upon which final predictions would be made
test.drop(['Username'],axis=1,inplace=True)
test=pd.get_dummies(test)
test.drop('Tag_x',axis=1,inplace=True)

In [19]:
#making predictions
preds = clf.predict(test.drop('ID',axis=1))

In [20]:
#making a dictionary which contains the ids and the corrosponding predictions
submission={
    'ID' : list(test['ID']),
    'Upvotes' : preds 
}
#writing the ids and corrosponding predictions to a dataframe
my_submission = pd.DataFrame(submission)
my_submission.head()

,ID,Upvotes
0,366953,201.6
1,71864,65.6
2,141692,27.1
3,316833,20.2
4,440445,401.3


In [21]:
# writing the dataframe to a csv file
my_submission.to_csv('final_submission(rfc)test.csv',index=False)